### Imports & constants

In [1]:
import torch
from torchvision.utils import make_grid
from torch.utils.data import DataLoader, Subset

from synthesis_dataset import SynthesisDataset

from net import Cycle
from bdcn import BDCN

import numpy as np

In [2]:
np.random.seed(42)

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

device = torch.device('cuda')
batch_size = 2

### Dataloaders

In [3]:
dataset = SynthesisDataset("..\simulation-synthesis\output\MLDataset", extension='.png')
dataset.modalities = ['img', 'outlines']

rtrain = Subset(dataset, range(0, int(0.9*len(dataset))))
rval = Subset(dataset, range(int(0.9*len(dataset)), len(dataset)))

train_loader = DataLoader(rtrain, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(rval, batch_size=batch_size, shuffle=False)

In [4]:
model = BDCN(pretrained=True).to(device)
model.train()
optimizer = torch.optim.SGD(params=model.parameters(), momentum=0.9, lr=1e-6, weight_decay=0.0002)

c:\Users\appel\Project\image-synthesis\bdcn.py:207: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.080)


In [5]:
for data in train_loader:
    optimizer.zero_grad()
    images = data['img'].to(device)
    labels = data['outlines'].to(device)
    out = model(images)
    loss = combined_loss(out, labels)
    loss.backward()
    batch_loss += loss.item()

11

torch.Size([2, 1, 128, 128])